<a href="https://colab.research.google.com/github/SanjayJanardhan-89/ApacheSparkHandsOn/blob/main/Unstructured_TextFile_Loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession

spark= SparkSession \
       .builder \
       .appName("OurSparkApp") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,686 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,793 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,773

*** Create a file in sample_data with the name "Context.txt" **bold text**

All information in this manual is based on the latest product information available at the time of publication. Due to continuous
improvements, there may be differences between the information provided in this manual and information related
to your motorcycle.
Always consult an authorized Royal Enfield dealer for the latest specifications, features etc. Royal Enfield reserves the right
to make production changes at any time without prior notice and without incurring any obligation to make the same or
similar changes to a motorcycle previously built or sold. All images shown are for reference to explain and need not to be
exactly the same on the model you own. Accessories and features may not be part of standard equipment. Technical specifications
are subject to change without prior notice at the sole discretion of Royal Enfield.
Please take care while disassembling and assembling the seats, and sheet metal parts, as any sharp edges will lead to
injuries.



In [17]:
from pyspark.sql.functions import split, lower, regexp_replace, explode


df = spark.read.text("sample_data/Content.txt")
df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                            |
+---------------------------------------------------------------------------------------------------------------------------------+
|All information in this manual is based on the latest product information available at the time of publication. Due to continuous|
|improvements, there may be differences between the information provided in this manual and information related                   |
|to your motorcycle.                                                                                                              |
|Always consult an authorized Royal Enfield dealer for the latest specifications, features etc. Royal Enfield reserves the right  |
|to make production changes at any time without prior notice and without inc

In [18]:
df_cleaned = df.withColumn( "clean_text"
                          , lower(regexp_replace(df["value"], "[^a-zA-Z\s]", ""))
                          )

# Tokenize (split into words)
df_tokens = df_cleaned.withColumn("words", split(df_cleaned["clean_text"], "\s+"))

df_tokens.select("words").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------+
|words                                                                                                                                                 |
+------------------------------------------------------------------------------------------------------------------------------------------------------+
|[all, information, in, this, manual, is, based, on, the, latest, product, information, available, at, the, time, of, publication, due, to, continuous]|
|[improvements, there, may, be, differences, between, the, information, provided, in, this, manual, and, information, related]                         |
|[to, your, motorcycle]                                                                                                                                |
|[always, consult, an, authorized, royal, enfield, dealer, for, the, latest, speci

In [16]:
df_word_count = (df_tokens
                  .select(explode(df_tokens["words"]).alias("word"))
                  .groupBy("word").count()\
                  .orderBy("count", ascending=False)
                )

df_word_count.show()

+--------------+-----+
|          word|count|
+--------------+-----+
|           the|   10|
|            to|    9|
|           and|    6|
|   information|    4|
|            be|    3|
|       enfield|    3|
|         royal|    3|
|       without|    3|
|           any|    3|
|            of|    3|
|            at|    3|
|specifications|    2|
|           not|    2|
|          time|    2|
|        latest|    2|
|            in|    2|
|          make|    2|
|    motorcycle|    2|
|        manual|    2|
|        notice|    2|
+--------------+-----+
only showing top 20 rows

